In [16]:
import collections
import random
import re
from collections import Counter
from itertools import islice

In [17]:
arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [18]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text
print(remove_diacritics("الْعَرَبِيّةُ"))

العربية


In [19]:
def remove_repeating_char(text):
    if 'الله' in text:
        return re.sub(r'(.)\1{2,}', r'\1\1', text)  # keep 2 repeats if 'الله' is present
    else:
        return re.sub(r'(.)\1+', r'\1', text)  # keep only 1 repeat otherwise

print(remove_repeating_char("الوووو ابرار"))
print(remove_repeating_char("الله"))

الو ابرار
الله


In [20]:
import string
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

print(remove_punctuations('هذه نص باللغة العربية! تحتوي على علامات الترقيم؛ وهي: ،.؟'))

هذه نص باللغة العربية تحتوي على علامات الترقيم وهي 


In [21]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

print(normalize_arabic(".إنّ الذين يتّبعون الصّيغ المعمّاة للكتابة، يتجنّبون الكلمات التي تتضمّن حروفاً مهمّلة."))

.انّ الذين يتّبعون الصّيغ المعمّاه للكتابه، يتجنّبون الكلمات التي تتضمّن حروفاً مهمّله.


In [22]:
def window(words_seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(words_seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

print(list(window("إنّ الذين يتّبعون الصّيغ المعمّاة للكتابة، يتجنّبون الكلمات التي تتضمّن حروفاً مهمّلة.".split())))

[('إنّ', 'الذين'), ('الذين', 'يتّبعون'), ('يتّبعون', 'الصّيغ'), ('الصّيغ', 'المعمّاة'), ('المعمّاة', 'للكتابة،'), ('للكتابة،', 'يتجنّبون'), ('يتجنّبون', 'الكلمات'), ('الكلمات', 'التي'), ('التي', 'تتضمّن'), ('تتضمّن', 'حروفاً'), ('حروفاً', 'مهمّلة.')]


In [23]:
def process_text(text, grams=False):
    clean_text = remove_diacritics(text)
    clean_text = remove_repeating_char(clean_text)
    clean_text = remove_punctuations(clean_text)
    clean_text = normalize_arabic(clean_text)
    if grams is False:
        return clean_text.split()
    else:
        tokens = clean_text.split()
        grams = list(window(tokens))
        grams = [' '.join(g) for g in grams]
        grams = grams + tokens
        return grams

In [24]:
def document_features(document, corpus_features):
    document_words = set(document) # create a set of unique words in the document
    features = {}
    for word in corpus_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

print(document_features("إخراج الكلمات المميزه الغير مكرره مكرره".split(), 
                        ["إخراج", "الكلمات", "المميزه", "الغير", "مكرره"]))

{'contains(إخراج)': True, 'contains(الكلمات)': True, 'contains(المميزه)': True, 'contains(الغير)': True, 'contains(مكرره)': True}


In [25]:
all_features = list()
texts = list()
data_labels = list()

positive_file = 'datasets/positive_tweets_arabic_20181207_300.txt'
negative_file = 'datasets/negative_tweets_arabic_20181207_300.txt'

n_grams_flag = False
min_freq = 13

In [26]:
with open(positive_file, encoding='utf-8') as tweets_file:
    for line in tweets_file:
        text_features = process_text(line, grams=n_grams_flag)
        all_features += text_features
        texts.append(text_features)
        data_labels.append('pos')

# read negative data
with open(negative_file, encoding='utf-8') as tweets_file:
    for line in tweets_file:
        text_features = process_text(line, grams=n_grams_flag)
        all_features += text_features
        texts.append(text_features)
        data_labels.append('neg')

In [27]:
print('data size', len(data_labels))
print('# of positive', data_labels.count('pos'))
print('# of negative', data_labels.count('neg'))

data size 21083
# of positive 6748
# of negative 14335


In [28]:
import nltk
from nltk.metrics.scores import f_measure

tweets = [(t, l) for t, l in zip(texts, data_labels)]
random.shuffle(tweets)
print('sample tweets')
for t in tweets[:10]: print(t)  # see the first 10 instances
print('\nall features', all_features[:20])
print('len(all_features):', len(all_features))

all_features_freq = nltk.FreqDist(w for w in all_features)
print(all_features_freq)
print('\nmost common features:', all_features_freq.most_common(20))
print('\nfreq of في: ', all_features_freq['في'])
thr = min_freq / len(all_features)
print('The minimum frequency threshold for features:', thr)

sample tweets
(['الجماعه', 'ف', 'السعوديه', 'جابوا', 'الڤيديوهات', 'لاحلام', 'من', 'الي', 'تحت', 'الحوض', 'و', 'انا', 'مبسوطه', 'اوي', 'بالي', 'بيحصلها', 'ده', '☺', '️'], 'pos')
(['صباح', 'النور', '😊'], 'pos')
(['يا', 'معلم', 'يا', 'استاز', '❤'], 'pos')
(['لاسف', 'عكاظ', 'كانت', 'محترمه', 'سابقا', 'والان', 'شوهوها', 'الطبالين', 'والراقصات', 'عكاظ', 'بالامس', 'فخمه', 'عكاظ', 'اليوم', 'فحمه', '✋', '🏻', '💚'], 'neg')
(['شو', 'عم', 'ضحك', 'انا', '😏'], 'neg')
(['الايام', 'القادمه', 'ستكون', 'بارده', '⛄', '️', 'كسوه', 'الشتاء', 'جاهزه', 'لتوزيع', 'الخيري', '🎒', 'تحوي', 'علي', 'شال', 'وشراب', 'وقفاز', 'وقبعه', '🧦', '•', 'مناسبه', 'لتوزيع', 'علي'], 'neg')
(['الايام', 'القادمه', 'ستكون', 'بارده', '⛄', '️', 'كسوه', 'الشتاء', 'جاهزه', 'لتوزيع', 'الخيري', '🎒', 'تحوي', 'علي', 'شال', 'وشراب', 'وقفاز', 'وقبعه', '🧦', '•', 'مناسبه', 'لتوزيع', 'علي'], 'neg')
(['•', 'يعلن', 'انتقال', 'كينو', 'لاتحاد', 'ثم', 'يرفض', '•', 'يوعد', 'جمهور', 'الاتحاد', 'بنجوم', 'عالمين', 'وينهي', 'عقد', 'المدرب', 'ويجلب', 'اس'

In [29]:
# remove features that have frequency below the threshold
my_features = set([word for word in all_features if all_features_freq.freq(word) > thr])
print('my_features:', my_features)
print(len(my_features), 'are kept out of', len(all_features))
print('features are selected')
print('------------------------------------')

print('sample features:')
print(list(my_features)[:10])
print('------------------------------------')

print('generating features for documents ...')
feature_sets = [(document_features(d, my_features), c) for (d, c) in tweets]


my_features: {'ومناسبه', 'والا', 'قبر', 'اني', 'دلني', 'اخي', 'ونس', 'سعودي', 'اعلان', 'لمن', 'غالي', 'قلوب', '2', 'وريفر', 'باليل', 'تخلت', 'وان', 'عندما', 'الفلسطينيه', 'ذ', 'فانت', 'جميل', '♵', 'الفيلم', 'اسف', 'القران', 'تخفيضات', 'فض', 'اسبوع', '💯', 'اختبار', 'الديربي', 'ايوه', 'خساره', 'فيهم', 'مسلسل', 'ت', 'اشياء', 'الافضل', 'هتفرحي', 'وعاد', 'والاماراتين', 'كلنا', 'تم', 'قاعده', '♨', 'ظن', 'اكون', 'شويه', 'وبعدها', 'شاهد', 'الرياضه', 'لسانك', 'القرار', 'القمر', 'يوجد', 'تعتبر', 'تحقيق', 'لهذا', 'ضحكوا', 'النفس', 'ادخل', 'سر', 'يغادر', '📱', 'النادي', 'انكساره', 'داءما', 'رضا', '👎', '😆', 'معاك', '✿', 'ٰ', 'تحوي', 'حلو', 'اس', 'هدف', 'فان', 'يقولون', 'تاني', 'الثقه', '😞', 'لعل', '⚠', 'يلي', '😷', 'هموم', 'الف', 'الوحيد', 'كتاب', 'شهر', 'تعليق', 'كنا', '﮼', 'الاحتجاج', 'سجن', 'فاضي', 'ايها', 'فلوس', 'لاي', 'س', 'بنفس', 'الزواج', '♳', 'اولا', '❋', 'عافيتها', 'بكرا', 'الجميل', 'مثلك', 'تنافي', 'موقع', 'صدق', '💗', 'دي', '📂', 'يلعن', '1', 'مشي', 'حد', 'هنالك', 'التغريدات', 'اضافي', 'الح

In [30]:
print('splitting documents into train and test ...')
print('data set size', len(data_labels))
train_percentage = 0.8
splitIndex = int(len(tweets) * train_percentage)
print('split index:', splitIndex)

random.shuffle(feature_sets)
train_set, test_set = feature_sets[:splitIndex], feature_sets[splitIndex:]
y_train = [l for t, l in train_set]
y_test = [l for t, l in test_set]

print('data set:', Counter(data_labels))
print('train:', Counter(y_train))
print('test:', Counter(y_test))

print('training NaiveBayes classifier ...')
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('training is done')

ref_sets = collections.defaultdict(set)
test_sets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    ref_sets[label].add(i)
    observed = classifier.classify(feats)
    test_sets[observed].add(i)

print('accuracy:', nltk.classify.accuracy(classifier, test_set))
print('positive f-score:', f_measure(ref_sets['pos'], test_sets['pos']))
print('negative f-score:', f_measure(ref_sets['neg'], test_sets['neg']))


classifier.show_most_informative_features(20)

splitting documents into train and test ...
data set size 21083
split index: 16866
data set: Counter({'neg': 14335, 'pos': 6748})
train: Counter({'neg': 11417, 'pos': 5449})
test: Counter({'neg': 2918, 'pos': 1299})
training NaiveBayes classifier ...
training is done
accuracy: 0.8868864121413327
positive f-score: 0.8266085059978189
negative f-score: 0.9160654583846561
Most Informative Features
             contains(😙) = True              pos : neg    =    499.3 : 1.0
       contains(المخفضه) = True              pos : neg    =    418.3 : 1.0
             contains(😚) = True              pos : neg    =    296.8 : 1.0
         contains(التزم) = True              neg : pos    =    146.2 : 1.0
             contains(⁧) = True              neg : pos    =    141.1 : 1.0
      contains(المنتجات) = True              pos : neg    =    139.9 : 1.0
           contains(نشر) = True              neg : pos    =    130.9 : 1.0
             contains(🐝) = True              neg : pos    =    108.7 : 1.0
   